In [5]:
import pandas as pd
import numpy as np

from gbdt import LightGBM

from sklearn.metrics import (
    accuracy_score,
    f1_score,
    confusion_matrix,
    classification_report,
    ConfusionMatrixDisplay,
)
import matplotlib.pyplot as plt

In [9]:
X = pd.read_csv("work/trn_curated_features.csv").drop(columns=["fname"])
Y = pd.read_csv("work/trn_curated_labels.csv").drop(columns=["fname"])
mask = (Y.sum(axis=1) == 1)  # 每行求和，等于1的表示只有一个标签
X = X[mask].reset_index(drop=True)
Y = Y[mask].reset_index(drop=True)

In [10]:
param = params = {
    'objective': 'binary',
    'metric': ['binary_logloss', 'auc'],
    'learning_rate': 0.05,
    'num_leaves': 15,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.9,
    'bagging_freq': 1,
    'min_data_in_leaf': 30,
    'lambda_l2': 1.0,
    'verbose': -1,
    'seed': 42,
}

lgbm = LightGBM(80, param)

In [37]:
X.shape

(4269, 2474)

In [51]:
train_X = X[:3269]
val_X = X[3269:3769]
train_Y = Y[:3269]
val_Y = Y[3269:3769]

test_X = X[3769:]
test_Y = Y[3769:]

In [52]:
lgbm.train(train_X,train_Y,val_X,val_Y)

  4%|▍         | 3/80 [00:03<01:14,  1.03it/s, f1_score=0]    /opt/anaconda3/envs/torch/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/torch/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/torch/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(averag

In [53]:
preds = np.zeros((500,80))
gt = np.zeros((500,80))
labels = list(Y.columns)
for i in range(len(labels)):
    bst = lgbm.bsts[i]
    thresh = lgbm.thresh[i]
    preds[:, i] = bst.predict(test_X)
    gt[:, i] = np.array(test_Y[labels[i]])

y_pred = np.zeros((500,))
y_true = np.zeros((500,))
for i in range(len(preds)):
    y_pred[i] = np.argmax(preds[i,:])
    y_true[i] = np.argmax(gt[i,:])

In [54]:
accuracy_score(y_true,y_pred)

0.482

In [43]:
preds = np.zeros((500,80))
gt = np.zeros((500,80))
labels = list(Y.columns)
for i in range(len(labels)):
    preds[:, i] = np.random.random((500,))
    gt[:, i] = np.array(val_Y[labels[i]])

y_pred = np.zeros((500,))
y_true = np.zeros((500,))
for i in range(len(preds)):
    y_pred[i] = np.argmax(preds[i,:])
    y_true[i] = np.argmax(gt[i,:])

accuracy_score(y_true,y_pred)

0.012